In [1]:
from src.lib import *
from src.transfomer import *

## Data Prepocessing

In [2]:
with open("./data/Shakespeare.txt",'r') as f:
    text = f.read()

In [3]:
chars = sorted(list(set(text)))

In [4]:
vocab_size = len(chars)

In [5]:
vocab_size

65

In [6]:
str_to_idx = {char:idx for idx,char in enumerate(chars)}
idx_to_str = {idx:char for idx,char in enumerate(chars)}

In [7]:
encode = lambda s :[str_to_idx[i] for i in s]
decode = lambda l : "".join([idx_to_str[i.item()] for i in l ])

In [8]:
text_encoded = encode(text)
text_encoded = torch.tensor(text_encoded)

In [9]:
p = 0.9
n = int(p * len(text_encoded))
train_data = text_encoded[0:n]
test_data = text_encoded[n:]

In [10]:
train_data.shape,test_data.shape

(torch.Size([1003854]), torch.Size([111540]))

## Embedding

In [11]:
B,T,C = 4,8,2
torch.tril(torch.rand(3,3))

tensor([[0.2821, 0.0000, 0.0000],
        [0.3401, 0.8726, 0.0000],
        [0.6796, 0.6610, 0.0130]])

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
model = Transformer(
    vocab_size=vocab_size,
    embed_size=384,
    heads=6,
    num_layers=6,
    max_len=1000,
    device=device
    dropout=0.2
)

TypeError: __init__() missing 1 required positional argument: 'device'

In [ ]:
model.fit(train_data,batch_size=8,block_size=16,n_iter=1000)